In [1]:
import os
import numpy as np 
import torch

from name_generator import create_name_file
from image_loader import image_loader_from_file

from pre_train import train_network
from feature_generator import feature_gen
from networks import encoder, decoder, adv_classifier

from z_encoder_train import train_z_encoder
from style import train_s_encoder

from utils import pickle_load
import params
# import load_data

from generate_glove_vector import generate_glove_vector


if __name__ == "__main__":

    ## Generate a file containing image/sketch location and corresponding label
    ## Each line of the file contain image/sketch's absolute path and space separated label 
    ## It is divided into train, val and test unit.
    if( not os.path.isfile(params.path_image_file_list) ):
        create_name_file( dataset_path = params.path_image_dataset,
                          path_class_list = params.path_class_list,
                          split = [0.8,0.05, 0.15],
                          dump_location = params.path_image_file_list )
    
    if( not os.path.isfile(params.path_sketchy_file_list) ):
        create_name_file( dataset_path = params.path_sketchy_dataset,
                          path_class_list = params.path_class_list,
                          split = [0.8,0.05, 0.15],
                          dump_location = params.path_sketchy_file_list )
    
    if( not os.path.isfile(params.path_quickdraw_file_list) ):
        create_name_file( dataset_path = params.path_quickdraw_dataset,
                          path_class_list = params.path_class_list,
                          split = [0.5,0.02, 0.48],
                          dump_location = params.path_quickdraw_file_list )

    ## Generate a image/sketch loader from the file generated above.
    loader_image = image_loader_from_file( file_name = params.path_image_file_list )
    loader_sketchy = image_loader_from_file( file_name = params.path_sketchy_file_list )
    loader_quick_draw = image_loader_from_file( file_name = params.path_quickdraw_file_list )

/home/adarsh/anaconda3/lib/python3.7/site-packages/torchvision/transforms/transforms.py:219: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
/home/adarsh/anaconda3/lib/python3.7/site-packages/torchvision/transforms/transforms.py:703: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


1
2
3
4


In [2]:
from network import ResNetFc
from preprocess import preprocess_image
model = ResNetFc( resnet_name = 'ResNet50', use_bottleneck = False,
                      bottleneck_dim= 256, new_cls= True, class_num = params.num_class)
    
if(params.gpu_flag):
    model.cuda(params.gpu_name)

In [3]:
image_list = []
for i,(img,_) in enumerate(loader_image.image_gen()):
    image_list.append(img)
    if(i==5):
        break
    

In [4]:
%%timeit
images = image_list[0]
images = preprocess_image( array = images,
                               split_type = 'train',
                               use_gpu = params.gpu_flag,
                               gpu_name = params.gpu_name )

The slowest run took 6.68 times longer than the fastest. This could mean that an intermediate result is being cached.
793 ms ± 590 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
from multiprocessing import Pool
array = []
def f(x,y):
    return x


with Pool(5) as p:
    a = p.map(f, [(1,2), (2,2), (3,2)],)
print(a)

TypeError: f() missing 1 required positional argument: 'y'

In [6]:
%%timeit
with Pool(5) as p:
    p.map(f, np.arange(100000))

1.1 s ± 185 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
map(f,np.arange(100000))

106 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [18]:
list(map(f,[1,2,3],[2,3,4]))

[1, 2, 3]

In [19]:
[0]*10

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]